In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
training_data = pd.read_csv("../input/digit-recognizer/train.csv")
testing_data = pd.read_csv("../input/digit-recognizer/test.csv")

In [3]:
print(testing_data.shape,training_data.shape)

(28000, 784) (42000, 785)


In [5]:
X_train = training_data.drop(labels = ["label"],axis = 1) 
Y_train = training_data["label"]
print(X_train.shape,Y_train.shape)

(42000, 784) (42000,)


In [6]:
X = X_train / 255.0
test = testing_data / 255.0

In [7]:
X = X.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [9]:
from keras.utils import to_categorical
y=to_categorical(Y_train, num_classes = 10)

In [13]:
y

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [14]:
X_train, X_val, Y_train, Y_val = train_test_split(X,y,test_size = 0.1,random_state=5)

In [20]:
classifer = Sequential()

classifer.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
classifer.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
classifer.add(MaxPool2D(pool_size=(2,2)))
classifer.add(Dropout(0.25))


classifer.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
classifer.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
classifer.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
classifer.add(Dropout(0.25))


classifer.add(Flatten())
classifer.add(Dense(256, activation = "relu"))
classifer.add(Dropout(0.5))
classifer.add(Dense(10, activation = "softmax"))

In [21]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
classifer.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.5,min_lr=0.00001)

In [22]:
Datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False, 
        rotation_range=10,  
        zoom_range = 0.1,  
        width_shift_range=0.1, 
        height_shift_range=0.1,  
        )  


Datagen.fit(X_train)

In [28]:
model = classifer.fit_generator(Datagen.flow(X_train,Y_train, batch_size= 100),
                              epochs = 30, validation_data = (X_val,Y_val),
                              verbose = 2)

Epoch 1/30
 - 12s - loss: 0.1446 - accuracy: 0.9569 - val_loss: 0.0540 - val_accuracy: 0.9848
Epoch 2/30
 - 11s - loss: 0.1046 - accuracy: 0.9690 - val_loss: 0.0385 - val_accuracy: 0.9883
Epoch 3/30
 - 12s - loss: 0.0871 - accuracy: 0.9746 - val_loss: 0.0391 - val_accuracy: 0.9890
Epoch 4/30
 - 11s - loss: 0.0763 - accuracy: 0.9778 - val_loss: 0.0441 - val_accuracy: 0.9879
Epoch 5/30
 - 11s - loss: 0.0686 - accuracy: 0.9800 - val_loss: 0.0358 - val_accuracy: 0.9914
Epoch 6/30
 - 11s - loss: 0.0639 - accuracy: 0.9813 - val_loss: 0.0564 - val_accuracy: 0.9850
Epoch 7/30
 - 11s - loss: 0.0591 - accuracy: 0.9825 - val_loss: 0.0342 - val_accuracy: 0.9910
Epoch 8/30
 - 12s - loss: 0.0595 - accuracy: 0.9834 - val_loss: 0.0313 - val_accuracy: 0.9907
Epoch 9/30
 - 11s - loss: 0.0593 - accuracy: 0.9830 - val_loss: 0.0313 - val_accuracy: 0.9917
Epoch 10/30
 - 11s - loss: 0.0563 - accuracy: 0.9837 - val_loss: 0.0433 - val_accuracy: 0.9929
Epoch 11/30
 - 12s - loss: 0.0596 - accuracy: 0.9828 - val_

In [30]:
classifer.evaluate(X_train,Y_train)

37800/37800 [==============================] - 3s 86us/step


[0.018210851330312975, 0.9947618842124939]

In [31]:
classifer.evaluate(X_val,Y_val)

4200/4200 [==============================] - 0s 88us/step


[0.04014485076832075, 0.9904761910438538]

In [33]:
classifer.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)         

In [40]:
import json

In [51]:
json_string = classifer.to_json()

In [52]:
with open('Digit_Recognizer1.json',"w") as json_file:
    json_file.write(json_string)

In [42]:
classifer.save_weights('AdvDG_Model.h5')

In [44]:
from keras.models import load_model
model = load_model('Digit_Recogniser.h5')